In [1]:
# Code by ShYy

# This script uses MIR-1K dataset, to create a Noise Added MIR-1K-NA.
import os, random
import numpy as np
import scipy.io.wavfile as wav

In [2]:
# Mix Audio and Noise as 75%Audio and 25%Noise
def mix_audio(data, noise):
    mix = np.add(0.75*data, 0.25*noise)
    out = np.array(mix)

    return out

# Set a start point from front 15s of the noise audios. The human voice waves are all shorter than 15s.
def create_noise_piece(noisedata, data):
    datalength = len(data)
    startpoint = random.randrange(0, 240000)         # 30s noise has 480000 sampled points at 16kHz

    outputdata = noisedata[startpoint:startpoint + datalength]

    return outputdata

In [4]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [19]:
cd /content/gdrive/My Drive/Data/

/content/gdrive/My Drive/Data


In [20]:
os.getcwd()

'/content/gdrive/My Drive/Data'

In [21]:

# ----- Main Function Start -----

# Directories
wavsDir = os.getcwd() + "/Wavs/"
noiseDir = os.getcwd() + "/Noises/"
noiseAddedDir = os.getcwd() + "/Training/NoiseAdded_urban/"
humanVoiceDir = os.getcwd() + "/Training/HumanVoices/"

# Noise File Name
whiteNoise = "airconditional2.wav"
brownianNoise = "carhorn2.wav"
pinkNoise = "engine2.wav"

# The length of the noise file. Three noise file are all 30s.
os.chdir(noiseDir)
tempNoiseData = wav.read(whiteNoise)
noiseLength = len(tempNoiseData)

# Get Noises
wNoiseRate, wNoise = wav.read(whiteNoise)
bNoiseRate, bNoise = wav.read(brownianNoise)
pNoiseRate, pNoise = wav.read(pinkNoise)

# Define Mixture of audio
WhiteNoiseMix = []
BrownianNoiseMix = []
PinkNoiseMix = []

# File Counter for Debugging
fileCounter = 0

# Enter /Wavs/ dir to start mixing processing
os.chdir(wavsDir)

In [22]:
# The wave files are combined with 2 channel, left for music and right for human voice.
# For each wave file, use the right channel and sum it with three types of noises.
for fileName in os.listdir(wavsDir):
    if fileName.endswith(".wav"):
        # Read the wave file
        wavFileRate, wavFile = wav.read(fileName)

        # Use the right channel
        rightChannel = wavFile
        humanVoice = np.array(rightChannel) #Human Voice 추출 우리는 이과정을 안해도 됨

        # Normalize human voice sample. Gain from the other channel, BGM channel. HumanVoice sample 정규화
        humanVoicePeak = max(abs(humanVoice))
        wavFilePeak = np.iinfo(wavFile.dtype).max
        gain = float(wavFilePeak)/humanVoicePeak
        humanVoiceNormalized = np.array(humanVoice * gain)

        # Create Mixtures
        print("Mixing " + fileName + " with Noises...")
        WhiteNoiseMix = np.array(mix_audio(humanVoiceNormalized, create_noise_piece(wNoise, humanVoiceNormalized)))
        BrownianNoiseMix = np.array(mix_audio(humanVoiceNormalized, create_noise_piece(bNoise, humanVoiceNormalized)))
        PinkNoiseMix = np.array(mix_audio(humanVoiceNormalized, create_noise_piece(pNoise, humanVoiceNormalized)))

        # Write mixture audio files in the training directory.
        os.chdir(noiseAddedDir)
        fName, extendName = os.path.splitext(fileName)
        wav.write(fName + "_rnoise" + extendName, wavFileRate, WhiteNoiseMix.astype(np.int16))
        wav.write(fName + "_hnoise" + extendName, wavFileRate, BrownianNoiseMix.astype(np.int16))
        wav.write(fName + "_inoise" + extendName, wavFileRate, PinkNoiseMix.astype(np.int16))

        # Write human voice audio to the directory for computing with Noise Added audios
        #os.chdir(humanVoiceDir)
        #wav.write(fName + "_voice" + extendName, wavFileRate, humanVoiceNormalized.astype(np.int16))

        # End and back to the Waves directory
        print("Finished Processing: " + fileName)
        os.chdir(wavsDir)

        # Counter++
        fileCounter = fileCounter + 1

print("Total Processed: " + str(fileCounter) + " file(s).")

Mixing tammy_1_07_voice.wav with Noises...
Finished Processing: tammy_1_07_voice.wav
Mixing amy_5_06_voice.wav with Noises...
Finished Processing: amy_5_06_voice.wav
Mixing ariel_3_03_voice.wav with Noises...
Finished Processing: ariel_3_03_voice.wav
Mixing geniusturtle_4_09_voice.wav with Noises...
Finished Processing: geniusturtle_4_09_voice.wav
Mixing abjones_4_04_voice.wav with Noises...
Finished Processing: abjones_4_04_voice.wav
Mixing Ani_5_03_voice.wav with Noises...
Finished Processing: Ani_5_03_voice.wav
Mixing heycat_2_07_voice.wav with Noises...
Finished Processing: heycat_2_07_voice.wav
Mixing Kenshin_4_09_voice.wav with Noises...
Finished Processing: Kenshin_4_09_voice.wav
Mixing fdps_1_12_voice.wav with Noises...
Finished Processing: fdps_1_12_voice.wav
Mixing bobon_3_03_voice.wav with Noises...
Finished Processing: bobon_3_03_voice.wav
Mixing fdps_1_06_voice.wav with Noises...
Finished Processing: fdps_1_06_voice.wav
Mixing leon_3_06_voice.wav with Noises...
Finished Pr

In [ ]:
humanVoiceNormalized

array([  0., -13., -37., ..., -50.,  24.,  88.])

In [ ]:
create_noise_piece(wNoise, humanVoiceNormalized)

array([ -1700. , -25162.5,  16487.5, ...,  80500. ,  74825. ,  30712.5])

In [ ]:
WhiteNoiseMix = np.array(mix_audio(humanVoiceNormalized, create_noise_piece(wNoise, humanVoiceNormalized)))
WhiteNoiseMix

array([ -22.75,   -7.5 ,  -44.75, ..., -793.  , -784.5 , -731.75])

In [ ]:
pNoise

array([0, 0, 0, ..., 0, 0, 0], dtype=int16)

In [ ]:
wNoiseRate

16000